# 1. Introducció

*League of Legends* (LoL) és un dels videojocs d’estratègia en equip més populars del món.  
Cada partida enfronta **dos equips de cinc jugadors** que competeixen per destruir la base rival.  
Abans de començar, cada jugador —anomenat *invocador*— escull un **campió** d’entre més de **140 personatges únics**, cadascun amb habilitats i rols diferents (guerrer, mag, suport, assassí...).

Durant la partida, els equips intenten aconseguir **objectius estratègics** (com ara torres, dracs o barons) que donen grans avantatges.  
Analitzar aquesta informació —els objectius assolits, les baixes, l’or guanyat o la durada de la partida— ens permet extreure patrons que ajuden a entendre **per què un equip guanya o perd**.

En aquest projecte treballarem amb dades extretes de **Kaggle**, provinents del conjunt *League of Legends Ranked Games*.  
Aquest conjunt conté desenes de milers de partides classificades i registra, per a cadascuna, **estadístiques detallades dels equips i dels jugadors**.

L’objectiu és construir un **model predictiu capaç d’estimar quin equip guanyarà** una partida a partir de les seves estadístiques i de la composició dels campions escollits.  
A més, aprofitarem aquesta anàlisi per comprendre **quins factors tenen més influència en la victòria** dins d’un joc tan complex i dinàmic com *League of Legends*.


# 2. Introducció al joc

Per entendre els passos que s’han seguit durant el desenvolupament del model predictiu, és important conèixer els **elements principals del joc** i com aquests estan representats a les dades.

*League of Legends* és un joc d’estratègia en equip on dos equips de cinc jugadors competeixen per destruir la base rival.  
Cada jugador controla un **campió**, un personatge amb habilitats úniques i un **rol específic** dins de l’equip.

### Els campions
Els **campions** són els protagonistes del joc.  
N’existeixen més de 140, cadascun amb un conjunt d’habilitats diferents (atac, defensa, suport, control, etc.).  
Abans de cada partida, cada jugador escull un campió i aquest passa a ser la seva peça principal durant tot el joc.

Els campions no són tots iguals ni mantenen sempre el mateix rendiment.  
*League of Legends* és un joc que s’actualitza de manera constant mitjançant **versions o “parches”**, que introdueixen canvis d’equilibri (anomenats *buffs* o *nerfs*).  
Això significa que un campió pot ser molt fort en una versió concreta i menys eficaç en la següent.  
A partir d’aquestes variacions, la comunitat i els analistes acostumen a calcular els **winrates** o percentatges de victòria, que reflecteixen com de sovint un campió guanya una partida.

Aquest projecte utilitza aquests conceptes de forma quantitativa: s’ha calculat per a cada campió un valor de **força mitjana històrica** (*winrate estimat*) que s’incorpora al model mitjançant tècniques d’“encoding”.  
D’aquesta manera, el model pot tenir en compte la qualitat de la composició d’equip més enllà de les estadístiques pures de la partida.

### Objectius del joc

Tot i que els campions són la clau de l’estratègia, l’objectiu final continua sent destruir el **Nexe (Nexus)** de l’equip rival. Per aconseguir-ho, cal avançar per les tres línies (*lanes*) i assolir una sèrie d’objectius:

#### Nexus
És l’estructura central i el principal objectiu del joc. Quan un equip destrueix el Nexe enemic, guanya la partida.  
La variable **`winner`** del nostre dataset indica quin equip ho ha aconseguit.

#### Inhibidors
Estructures situades a la base que, en ser destruïdes, fan que els minions de l’equip rival es tornin molt més forts.  
Estan representats per les columnes **`*_firstinhib`** i **`*_inhibkills`**.

#### Torres
Estructures defensives que protegeixen les línies i la base.  
Les columnes **`*_firsttower`** i **`*_towerkills`** reflecteixen quin equip ha destruït la primera torre i el nombre total de torres destruïdes.

#### Herald del Rift
Monstre èpic que apareix al principi de la partida.  
L’equip que el derrota obté un objecte que pot utilitzar per fer-lo aparèixer i ajudar a destruir torres enemigues.  
Apareix a les columnes **`*_firstharry`** i **`*_harrykills`**.

#### Drac
Monstre èpic que apareix periòdicament i atorga millores permanents a les estadístiques de l’equip que el derrota.  
Les columnes **`*_firstdragon`** i **`*_dragonkills`** recullen aquesta informació.

#### Baró Nashor
És el monstre més important i ofereix un gran avantatge a l’equip que el derrota, augmentant el poder dels jugadors i dels seus minions.  
Es troba representat per **`*_firstbaron`** i **`*_baronkills`**.


En resum, el joc combina dos tipus de factors:  
- **Estratègics**, com els objectius aconseguits durant la partida (torres, dracs, barons...).  
- **Tàctics**, com la composició de campions i el seu rendiment històric.  

El nostre model de predicció utilitza tots aquests elements per intentar identificar quins patrons augmenten la probabilitat de victòria d’un equip.


# Predicció de l’equip guanyador a *League of Legends*

Aquest projecte utilitza un conjunt de **7 datasets** del joc *League of Legends (LoL)* amb l’objectiu de **predir quin equip guanyarà una partida** a partir de les seves estadístiques i la composició dels campions.

### Datasets originals i finalitat

1. **matches.csv**  
   Conté informació general de cada partida (durada, versió, temporada...).

2. **teamstats.csv**  
   Estadístiques a nivell d’equip: nombre de torres, dracs, barons, inhibidors, etc.

3. **teambans.csv**  
   Llista dels campions que cada equip ha prohibit (*bans*).  
   *(No s’utilitza al model principal, però podria servir per a anàlisis futures).*

4. **participants.csv**  
   Detalla els jugadors i els campions utilitzats a cada partida (10 per partida).

5. **stats1.csv / stats2.csv**  
   Contenen estadístiques individuals de cada jugador (kills, morts, or guanyat, minions, etc.).  
   Es combinen per tenir tots els jugadors.

6. **champs.csv**  
   Llista de campions disponibles (nom i identificador).


### Com s’han combinat

1. S’han unit `matches`, `teamstats`, `stats1` i `stats2` per crear un dataset amb estadístiques **per partida i equip** (`lol_clean.csv`).

2. Després s’ha afegit informació de **composició de campions** a través de `participants` i `champs`, aplicant un **Target Encoding** que resumeix la força mitjana dels campions de cada equip sense generar milers de columnes.

3. El resultat final és `lol_final.csv`, amb una fila per partida i columnes per a cada equip (`blue_*` i `red_*`), més la variable objectiu `winner`.

### Objectiu
Predir la variable:
> `winner` → 1 si guanya l’equip blau, 0 si guanya l’equip vermell

a partir de:
- Estadístiques de la partida (kills, or, objectius...)  
- Característiques derivades de la composició (mitjana i dispersió del rendiment dels campions)



## Com s’ha generat el dataset final `lol_final.csv`

Per arribar al dataset final amb què treballarem, hem combinat i processat **tots els fitxers originals** del conjunt de dades de *League of Legends*.

### Integració de dades bàsiques
Primer hem combinat els arxius principals (`matches`, `teamstats`, `stats1`, `stats2`, `participants`, `champs`) per construir un dataset amb **una fila per partida**, on cada equip (blau i vermell) té les seves estadístiques.

- S’han unit les dades dels **jugadors** amb les seves **estadístiques individuals** (`stats1` i `stats2`), i posteriorment s’han agregat a nivell d’equip (suma de kills, or, assistències, etc.).
- S’ha inferit l’equip de cada jugador (`teamid=100` per al blau, `200` per al vermell) a partir del número de jugador.
- S’han afegit els **objectius d’equip** (torres, dracs, barons...) des de `teamstats` i la **informació de la partida** (`durada`, `versió`, `temporada`) des de `matches`.
- Finalment, s’han combinat les estadístiques de l’equip blau i de l’equip vermell en una sola fila i s’ha creat la variable **`winner`**, que val 1 si guanya l’equip blau i 0 si guanya el vermell.

El resultat d’aquest procés és el fitxer `lol_clean.csv`, amb totes les estadístiques bàsiques de cada partida.

### Afegir informació sobre la composició d’equips
A continuació, s’ha enriquit el dataset amb informació sobre **els campions seleccionats** a cada partida, utilitzant `participants.csv`.

Per no crear milers de columnes (una per campió), s’ha aplicat un mètode de **Target Encoding**:
- S’ha calculat el **winrate mitjà de cada campió** (probabilitat de victòria històrica).
- Per a cada equip s’ha agregat el rendiment mitjà, màxim, mínim i desviació estàndard dels cinc campions triats.
- S’han afegit també les diferències entre els valors mitjans del blau i del vermell.

Això genera noves variables com `blue_te_mean`, `red_te_max` o `te_mean_diff`, que representen la **força estimada de la composició de cada equip**.

El resultat final és el fitxer **`lol_final.csv`**, que conté:
- Les estadístiques generals de partida (`lol_clean.csv`)
- + Les característiques de composició d’equip (Target Encoding)
- + La variable objectiu `winner`

Aquest fitxer és el que utilitzarem per fer l’anàlisi exploratòria i entrenar els models de predicció.
